In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=42f254dbc3d7200f2008fb4f62c3c2b13668d04d25810da98ff50d9805cccbd2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
import os
os.chdir('/content/drive/MyDrive/cosmetic')

In [ ]:
new_df = spark.read.option('header', True).option('inferSchema', True).csv('/content/drive/MyDrive/cosmetic/new.csv')
total = new_df.createOrReplaceTempView('total')

In [ ]:
# 컬럼 확인
item = spark.sql("""
SELECT *
FROM total
limit 10;
""")
item.show()

+-------------------+----------+----------+-------------------+-------------+------+-----+-------+------------+
|         event_time|event_type|product_id|        category_id|category_code| brand|price|user_id|user_session|
+-------------------+----------+----------+-------------------+-------------+------+-----+-------+------------+
|2020-01-14 16:15:21|      view|   5865526|1487580008447738866|         null|   cnd| 10.0| 465496|           1|
|2020-01-14 16:25:35|      view|   5769989|1487580008447738866|         null|   cnd| 10.0| 465496|           2|
|2020-01-14 16:27:31|      view|   5865524|1487580008447738866|         null|   cnd| 10.0| 465496|           2|
|2019-12-22 12:50:22|      view|   5746011|1487580009051717646|         null|runail|34.92|2963072|          38|
|2019-12-22 12:50:45|      view|   5707747|1487580009051717646|         null|  null|73.02|2963072|          38|
|2019-12-22 12:50:58|      view|   5746011|1487580009051717646|         null|runail|34.92|2963072|      

In [ ]:
# 컬럼 확인
result = spark.sql("""
SELECT count(distinct user_id)
FROM new
""")
result.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                1639358|
+-----------------------+



In [ ]:
# 고객구분 (0회/1회/2회 구매자)
# with절 -> purch_cnt(구매한 회원)
result = spark.sql("""
WITH purch_cnt AS (
  SELECT user_id
			 , COUNT(DISTINCT user_session) AS session_cnt
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
)
SELECT (SELECT COUNT(DISTINCT user_id) FROM new) AS total_cnt
		 , (SELECT COUNT(DISTINCT user_id) FROM new) - (SELECT COUNT(user_id) FROM purch_cnt) AS zero
		 , (SELECT COUNT(user_id) FROM purch_cnt WHERE session_cnt=1) AS once
		 , (SELECT COUNT(user_id) FROM purch_cnt WHERE session_cnt>=2) AS repurch
""")
result.show()

+---------+-------+-----+-------+
|total_cnt|   zero| once|repurch|
+---------+-------+-----+-------+
|  1639358|1528840|86607|  23911|
+---------+-------+-----+-------+



## 1. 코호트 분석: 코호트별 매출 확인

In [ ]:
# try 1
# 매출을 COUNT(DISTINCT user_id)로 나눔.
# 고객당 평균 매출 = 코호트 매출 / 코호트 내 고객수

result = spark.sql("""
WITH customers AS (
  SELECT user_id
       ,MIN(event_time) AS first_order_day
       ,MAX(event_time) AS last_order_day
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), records_month AS (
  SELECT to.user_id
      ,DATE_FORMAT(first_order_day, 'yyyy-MM-01') as first_order_month
      ,DATE_FORMAT(cu.first_order_day, 'yyyy-MM-dd') AS first_order_date
      ,DATE_FORMAT(cu.last_order_day, 'yyyy-MM-dd') AS last_order_date
      ,DATE_FORMAT(event_time, 'yyyy-MM-dd') as order_date
      ,price
  FROM total as to
    INNER JOIN customers cu ON to.user_id = cu.user_id AND event_type = 'purchase')
SELECT first_order_month
     ,ROUND(sum(price),2) / COUNT(DISTINCT user_id) day0
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 25)
                          AND order_date < DATE_ADD(first_order_date, 50) THEN price END),2) / COUNT(DISTINCT user_id) day25
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 50)
                          AND order_date < DATE_ADD(first_order_date, 75) THEN price END),2) / COUNT(DISTINCT user_id) day50
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 75)
                          AND order_date < DATE_ADD(first_order_date, 100) THEN price END),2) / COUNT(DISTINCT user_id) day75
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 100)
                          AND order_date < DATE_ADD(first_order_date, 125) THEN price END),2) / COUNT(DISTINCT user_id) day100
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 125)
                          AND order_date < DATE_ADD(first_order_date, 150) THEN price END),2) / COUNT(DISTINCT user_id) day125
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 150)
                          AND order_date < DATE_ADD(first_order_date, 175) THEN price END),2) / COUNT(DISTINCT user_id) day150

FROM records_month
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+------------------+--------------------+-------------------+------------------+------------------+-----------------+--------------------+
|first_order_month|              day0|               day25|              day50|             day75|            day100|           day125|              day150|
+-----------------+------------------+--------------------+-------------------+------------------+------------------+-----------------+--------------------+
|       2019-10-01| 82.44534702274667|  10.061702119400667|   6.62087415573325| 6.174474419687912| 6.565313252076702|3.432294852884093|0.027665553916621382|
|       2019-11-01|59.991043309293374|   3.802795485968387|   4.35885953439707|3.9113766301707713|1.0517136878293039|             null|                null|
|       2019-12-01|46.174096030859815|  3.3481519642675868| 2.4661019185869453|0.5556811491219166|              null|             null|                null|
|       2020-01-01| 47.17331473050375|    2.86712243599158

In [ ]:
# try 2
# 고객당 평균 매출 (반올림)

result = spark.sql("""
WITH customers AS (
  SELECT user_id
       ,MIN(event_time) AS first_order_day
       ,MAX(event_time) AS last_order_day
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), records_month AS (
  SELECT to.user_id
      ,DATE_FORMAT(first_order_day, 'yyyy-MM-01') as first_order_month
      ,DATE_FORMAT(cu.first_order_day, 'yyyy-MM-dd') AS first_order_date
      ,DATE_FORMAT(cu.last_order_day, 'yyyy-MM-dd') AS last_order_date
      ,DATE_FORMAT(event_time, 'yyyy-MM-dd') as order_date
      ,price
  FROM total as to
    INNER JOIN customers cu ON to.user_id = cu.user_id AND event_type = 'purchase')
SELECT first_order_month
     ,ROUND(sum(price) / COUNT(DISTINCT user_id),2) day0
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 25)
                          AND order_date < DATE_ADD(first_order_date, 50) THEN price END) / COUNT(DISTINCT user_id),2) day25
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 50)
                          AND order_date < DATE_ADD(first_order_date, 75) THEN price END) / COUNT(DISTINCT user_id),2) day50
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 75)
                          AND order_date < DATE_ADD(first_order_date, 100) THEN price END) / COUNT(DISTINCT user_id),2) day75
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 100)
                          AND order_date < DATE_ADD(first_order_date, 125) THEN price END) / COUNT(DISTINCT user_id),2) day100
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 125)
                          AND order_date < DATE_ADD(first_order_date, 150) THEN price END) / COUNT(DISTINCT user_id),2) day125
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 150)
                          AND order_date < DATE_ADD(first_order_date, 175) THEN price END) / COUNT(DISTINCT user_id),2) day150

FROM records_month
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+------------------+-----+-----+-----+------+------+------+
|first_order_month|              day0|day25|day50|day75|day100|day125|day150|
+-----------------+------------------+-----+-----+-----+------+------+------+
|       2019-10-01| 82.44534702274667|10.06| 6.62| 6.17|  6.57|  3.43|  0.03|
|       2019-11-01|59.991043309293374|  3.8| 4.36| 3.91|  1.05|  null|  null|
|       2019-12-01|46.174096030859815| 3.35| 2.47| 0.56|  null|  null|  null|
|       2020-01-01| 47.17331473050375| 2.87| 0.21| null|  null|  null|  null|
|       2020-02-01|41.930012873614686| 0.04| null| null|  null|  null|  null|
+-----------------+------------------+-----+-----+-----+------+------+------+



In [ ]:
#코호트별 기간 매출액
result = spark.sql("""
WITH customers AS (
  SELECT user_id
       ,MIN(event_time) AS first_order_day
       ,MAX(event_time) AS last_order_day
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), records_month AS (
  SELECT to.user_id
      ,DATE_FORMAT(first_order_day, 'yyyy-MM-01') as first_order_month
      ,DATE_FORMAT(cu.first_order_day, 'yyyy-MM-dd') AS first_order_date
      ,DATE_FORMAT(cu.last_order_day, 'yyyy-MM-dd') AS last_order_date
      ,DATE_FORMAT(event_time, 'yyyy-MM-dd') as order_date
      ,price
  FROM total as to
    INNER JOIN customers cu ON to.user_id = cu.user_id AND event_type = 'purchase')

SELECT first_order_month
     ,ROUND(sum(price),2) day0
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 25)
                          AND order_date < DATE_ADD(first_order_date, 50) THEN price END),2) day25
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 50)
                          AND order_date < DATE_ADD(first_order_date, 75) THEN price END),2) day50
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 75)
                          AND order_date < DATE_ADD(first_order_date, 100) THEN price END),2) day75
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 100)
                          AND order_date < DATE_ADD(first_order_date, 125) THEN price END),2) day100
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 125)
                          AND order_date < DATE_ADD(first_order_date, 150) THEN price END),2) day125
      ,ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 150)
                          AND order_date < DATE_ADD(first_order_date, 175) THEN price END),2) day150

FROM records_month
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+----------+---------+---------+---------+--------+--------+------+
|first_order_month|      day0|    day25|    day50|    day75|  day100|  day125|day150|
+-----------------+----------+---------+---------+---------+--------+--------+------+
|       2019-10-01|2123957.03|259209.57|170566.96|159066.81|169135.6|88422.78|712.72|
|       2019-11-01|1605420.31|101766.61|116647.44|104672.35|28144.91|    null|  null|
|       2019-12-01| 909722.04| 65965.29| 48587.14| 10948.03|    null|    null|  null|
|       2020-01-01|  963609.3| 58566.71|  4304.59|     null|    null|    null|  null|
|       2020-02-01| 749121.61|   762.76|     null|     null|    null|    null|  null|
+-----------------+----------+---------+---------+---------+--------+--------+------+



In [ ]:
# final 수정본
# 고객당 평균 매출 = 각 구간별 인원수 / 구간별 총매출
result = spark.sql("""
WITH customers AS (
  SELECT user_id
       , MIN(event_time) AS first_order_day
       , MAX(event_time) AS last_order_day
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), records_month AS (
  SELECT to.user_id
      , DATE_FORMAT(first_order_day, 'yyyy-MM-01') as first_order_month
      , DATE_FORMAT(cu.first_order_day, 'yyyy-MM-dd') AS first_order_date
      , DATE_FORMAT(cu.last_order_day, 'yyyy-MM-dd') AS last_order_date
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') as order_date
      , price
  FROM total as to
    INNER JOIN customers cu ON to.user_id = cu.user_id AND event_type = 'purchase'
)

SELECT first_order_month
     , ROUND(SUM(price) / COUNT(DISTINCT user_id), 2) as day0
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 25)
                          AND order_date < DATE_ADD(first_order_date, 50) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 25)
                          AND order_date < DATE_ADD(first_order_date, 50) THEN user_id END), 2) as day25
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 50)
                          AND order_date < DATE_ADD(first_order_date, 75) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 50)
                          AND order_date < DATE_ADD(first_order_date, 75) THEN user_id END), 2) as day50
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 75)
                          AND order_date < DATE_ADD(first_order_date, 100) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 75)
                          AND order_date < DATE_ADD(first_order_date, 100) THEN user_id END), 2) as day75
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 100)
                          AND order_date < DATE_ADD(first_order_date, 125) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 100)
                          AND order_date < DATE_ADD(first_order_date, 125) THEN user_id END), 2) as day100
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 125)
                          AND order_date < DATE_ADD(first_order_date, 150) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 125)
                          AND order_date < DATE_ADD(first_order_date, 150) THEN user_id END), 2) as day125
     , ROUND(SUM(CASE WHEN order_date >= DATE_ADD(first_order_date, 150)
                          AND order_date < DATE_ADD(first_order_date, 175) THEN price END) / COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 150)
                          AND order_date < DATE_ADD(first_order_date, 175) THEN user_id END), 2) as day150
FROM records_month
GROUP BY first_order_month
ORDER BY first_order_month;

""")
result.show()

+-----------------+-----+-----+-----+-----+------+------+------+
|first_order_month| day0|day25|day50|day75|day100|day125|day150|
+-----------------+-----+-----+-----+-----+------+------+------+
|       2019-10-01|82.45|63.44|56.61|58.29| 63.92| 63.66| 50.91|
|       2019-11-01|59.99|48.16|52.28|55.12| 51.83|  null|  null|
|       2019-12-01|46.17|47.84|47.08|45.24|  null|  null|  null|
|       2020-01-01|47.17|52.91|57.39| null|  null|  null|  null|
|       2020-02-01|41.93|34.67| null| null|  null|  null|  null|
+-----------------+-----+-----+-----+-----+------+------+------+



In [ ]:
result = spark.sql("""
WITH purchase_stats AS (
  SELECT DISTINCT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), total_preprocessed AS (
  SELECT t.user_id
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') order_date
       , ps.first_order_date
       , DATE_FORMAT(ps.first_order_date, 'yyyy-MM-01') first_order_month
       , t.price
  FROM total t
       INNER JOIN purchase_stats ps ON t.user_id = ps.user_id AND t.event_type = 'purchase'
)

SELECT first_order_month
     , ROUND(SUM(price) / COUNT(DISTINCT user_id), 2) day0
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 1) AND DATE_ADD(first_order_month, 25) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 1) AND DATE_ADD(first_order_month, 25)
       THEN user_id END), 2) day1_25
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 26) AND DATE_ADD(first_order_month, 50) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 26) AND DATE_ADD(first_order_month, 50)
       THEN user_id END), 2) day26_50
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 51) AND DATE_ADD(first_order_month, 75) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 51) AND DATE_ADD(first_order_month, 75)
       THEN user_id END), 2) day51_75
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 76) AND DATE_ADD(first_order_month, 100) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 76) AND DATE_ADD(first_order_month, 100)
       THEN user_id END), 2) day76_100
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 101) AND DATE_ADD(first_order_month, 125) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 101) AND DATE_ADD(first_order_month, 125)
       THEN user_id END), 2) day101_125
     , ROUND(SUM(CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 126) AND DATE_ADD(first_order_month, 150) THEN price END) /
       COUNT(DISTINCT CASE WHEN order_date BETWEEN DATE_ADD(first_order_month, 126) AND DATE_ADD(first_order_month, 150)
       THEN user_id END), 2) day126_150
FROM total_preprocessed
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+-----+-------+--------+--------+---------+----------+----------+
|first_order_month| day0|day1_25|day26_50|day51_75|day76_100|day101_125|day126_150|
+-----------------+-----+-------+--------+--------+---------+----------+----------+
|       2019-10-01|82.45|  46.06|   54.86|   59.62|    54.37|     63.01|      64.1|
|       2019-11-01|59.99|  44.98|   44.82|   48.51|    54.75|     53.48|      null|
|       2019-12-01|46.17|  38.54|   41.95|   47.37|    44.53|      null|      null|
|       2020-01-01|47.17|  41.74|   45.74|   48.39|     null|      null|      null|
|       2020-02-01|41.93|  41.88|   38.25|    null|     null|      null|      null|
+-----------------+-----+-------+--------+--------+---------+----------+----------+



In [ ]:
result = spark.sql("""
WITH purchase_stats AS (
  SELECT DISTINCT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), total_preprocessed AS (
  SELECT t.user_id
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') order_date
       , ps.first_order_date
       , DATE_FORMAT(ps.first_order_date, 'yyyy-MM-01') first_order_month
       , t.price
  FROM total t
       INNER JOIN purchase_stats ps ON t.user_id = ps.user_id AND t.event_type = 'purchase'
)

SELECT *
FROM total_preprocessed
""")
result.show()

+--------+----------+----------------+-----------------+-----+
| user_id|order_date|first_order_date|first_order_month|price|
+--------+----------+----------------+-----------------+-----+
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 6.35|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 1.59|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 1.59|
|43713532|2019-10-13|      2019-10-13|       2019-10-01|  1.9|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 3.97|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 3.97|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 3.97|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 3.57|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 5.56|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 6.27|
|43713532|2019-10-13|      2019-10-13|       2019-10-01|10.95|
|43713532|2019-10-13|      2019-10-13|       2019-10-01| 1.59|
|43713532|2019-10-13|      2019-10-13|       2019-10-01

## 2. 리텐션 분석

In [ ]:
# 1단계
result = spark.sql("""
SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
""")
result.show()

+---------+----------------+---------------+
|  user_id|first_order_date|last_order_date|
+---------+----------------+---------------+
| 43713532|      2019-10-13|     2019-10-13|
|129675214|      2020-01-03|     2020-01-03|
|189390340|      2020-02-07|     2020-02-07|
|204142009|      2020-01-20|     2020-01-20|
|207395269|      2019-11-08|     2020-01-24|
|221988388|      2019-12-24|     2019-12-24|
|230350829|      2019-10-12|     2020-01-08|
|231023829|      2019-12-18|     2020-01-27|
|231048129|      2019-11-14|     2020-02-23|
|233169997|      2019-12-22|     2019-12-22|
|233321490|      2019-11-07|     2019-12-06|
|234024900|      2019-10-15|     2020-02-12|
|235005368|      2019-12-12|     2020-01-08|
|235215029|      2020-01-27|     2020-01-27|
|236532316|      2020-02-01|     2020-02-24|
|237317205|      2019-11-06|     2019-11-06|
|246980024|      2019-11-02|     2019-11-02|
|264865866|      2019-10-22|     2020-01-27|
|269428285|      2019-10-22|     2020-02-29|
|274114452

In [ ]:
# 2단계
# processed table만들기
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT *
FROM processed;

""")
result.show()

+--------+-----------------+----------+----------------+
| user_id|first_order_month|order_date|first_order_date|
+--------+-----------------+----------+----------------+
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|201

In [ ]:
# 2단계
# processed table만들기
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT count(distinct user_id)
FROM processed;

""")
result.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 110518|
+-----------------------+



In [ ]:
result = spark.sql("""


""")
result.show()

In [ ]:
result = spark.sql("""


""")
result.show()

## 리텐션 (cycle/n-day =25일)
- with절 구문 사용
  1단계 stats: 구매세션을 id별로 그룹핑하고, 날짜형태를 2020-10-01 로 변경
  2단계 processed: 전체 구매리스트에서
- 의문점: 애초에 구매횟수에 대한 조건을 건게 아니고
- 구매이력이 있는 id 대상으로 리텐션분석한거라 분석제목이 기존고객의 리텐션이라기보다 그냥 리텐션 아닌가..?

In [ ]:
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT first_order_month
     , COUNT(DISTINCT user_id) AS day0
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 25) AND order_date < DATE_ADD(first_order_date, 50) THEN user_id END) / COUNT(DISTINCT user_id) AS day25
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 50) AND order_date < DATE_ADD(first_order_date, 75) THEN user_id END) / COUNT(DISTINCT user_id) AS day50
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 75) AND order_date < DATE_ADD(first_order_date, 100) THEN user_id END) / COUNT(DISTINCT user_id) AS day75
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 100) AND order_date < DATE_ADD(first_order_date, 125) THEN user_id END) / COUNT(DISTINCT user_id) AS day100
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 125) AND order_date < DATE_ADD(first_order_date, 150) THEN user_id END) / COUNT(DISTINCT user_id) AS day125
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 150) AND order_date < DATE_ADD(first_order_date, 175) THEN user_id END) / COUNT(DISTINCT user_id) AS day150
FROM processed
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

In [ ]:
# try 1
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM new
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , first_order_date
  FROM new t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT first_order_month
     , COUNT(DISTINCT user_id) AS day0
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 25) AND order_date < DATE_ADD(first_order_date, 50) THEN user_id END) / COUNT(DISTINCT user_id) AS day25
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 50) AND order_date < DATE_ADD(first_order_date, 75) THEN user_id END) / COUNT(DISTINCT user_id) AS day50
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 75) AND order_date < DATE_ADD(first_order_date, 100) THEN user_id END) / COUNT(DISTINCT user_id) AS day75
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 100) AND order_date < DATE_ADD(first_order_date, 125) THEN user_id END) / COUNT(DISTINCT user_id) AS day100
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 125) AND order_date < DATE_ADD(first_order_date, 150) THEN user_id END) / COUNT(DISTINCT user_id) AS day125
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 150) AND order_date < DATE_ADD(first_order_date, 175) THEN user_id END) / COUNT(DISTINCT user_id) AS day150
FROM processed
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+-----+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|first_order_month| day0|               day25|               day50|               day75|             day100|              day125|              day150|
+-----------------+-----+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|       2019-10-01|25762| 0.15860569831534818| 0.11695520534120021| 0.10593121652045649|0.10270941697073209|0.053916621380327615|5.434360686282121E-4|
|       2019-11-01|26761| 0.07895818541908001| 0.08336758716042002| 0.07096147378648032| 0.0202907215724375|                 0.0|                 0.0|
|       2019-12-01|19702| 0.06999289412242411| 0.05238046898792001|0.012283016952593645|                0.0|                 0.0|                 0.0|
|       2020-01-01|20427|0.054192979879571154|0.003671611102951...|                 0.0|      

In [ ]:
# try 2
# 첫구매 월별 구매자 수 확인
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM new
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , first_order_date
  FROM new t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT first_order_month
     , COUNT(DISTINCT user_id) AS day0
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 25) AND order_date < DATE_ADD(first_order_date, 50) THEN user_id END) AS day25
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 50) AND order_date < DATE_ADD(first_order_date, 75) THEN user_id END) AS day50
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 75) AND order_date < DATE_ADD(first_order_date, 100) THEN user_id END) AS day75
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 100) AND order_date < DATE_ADD(first_order_date, 125) THEN user_id END) AS day100
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 125) AND order_date < DATE_ADD(first_order_date, 150) THEN user_id END) AS day125
     , COUNT(DISTINCT CASE WHEN order_date >= DATE_ADD(first_order_date, 150) AND order_date < DATE_ADD(first_order_date, 175) THEN user_id END) AS day150
FROM processed
GROUP BY first_order_month
ORDER BY first_order_month

""")
result.show()

+-----------------+-----+-----+-----+-----+------+------+------+
|first_order_month| day0|day25|day50|day75|day100|day125|day150|
+-----------------+-----+-----+-----+-----+------+------+------+
|       2019-10-01|25762| 4086| 3013| 2729|  2646|  1389|    14|
|       2019-11-01|26761| 2113| 2231| 1899|   543|     0|     0|
|       2019-12-01|19702| 1379| 1032|  242|     0|     0|     0|
|       2020-01-01|20427| 1107|   75|    0|     0|     0|     0|
|       2020-02-01|17866|   22|    0|    0|     0|     0|     0|
+-----------------+-----+-----+-----+-----+------+------+------+

